# Importing Packages

In [2]:
import random
import gym
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
8
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# import gymnasium as gym
import numpy as np

# Creating a Model

In [3]:
# import necessary modules from keras
from keras.layers import Dense
from keras.models import Sequential
import keras
from keras.models import load_model

model = Sequential()
model.add(Dense(units=200,input_dim=80*80, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(units=1, activation='sigmoid', kernel_initializer='RandomNormal'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model = keras.models.load_model('./weights/og_weight_test.keras')
# model.load_weights("./weights/my_model_weights.h5")

# Preprocessing Functions

In [4]:
import matplotlib.pyplot as plt

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] 
  I = I[::2,::2,0] 
  I[I == 144] = 0 
  I[I == 109] = 0 
  I[I != 0] = 1 
  # with np.printoptions(threshold=np.inf):
  #   print(I)
  return I.astype(float).ravel()

def discount_rewards(r, gamma):
  """ take 1D float array of rewards and compute discounted reward """
  r = np.array(r)
  discounted_r = np.zeros_like(r)
  running_add = 0

  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  discounted_r -= np.mean(discounted_r) 
  discounted_r /= np.std(discounted_r) 
  print(discounted_r)
  return discounted_r

# Defining the Agent

In [5]:
import numpy as np
import gym

# gym initialization
env = gym.make("ALE/Pong-v5", render_mode="human")
observation = env.reset()
prev_input = None

# plt.imshow(observation[0])

# Macros
UP_ACTION = 2
DOWN_ACTION = 3

# Hyperparameters
gamma = 0.99

# initialization of variables used in the main loop
x_train, y_train, rewards = [],[],[]
reward_sum = 0
episode_nb = 0

print(observation[0].shape)

(210, 160, 3)


# Training

In [5]:


# main loop
while (True):
    env.render()
    # preprocess the observation, set input as difference between images
#     print(observation[0].shape)
    cur_input = prepro(observation[0])
    x = cur_input - prev_input if prev_input is not None else np.zeros(80 * 80)
    prev_input = cur_input
    
    # forward the policy network and sample action according to the proba distribution
    proba = model.predict(np.expand_dims(x, axis=1).T)
    action = UP_ACTION if np.random.uniform() < proba else DOWN_ACTION
    y = 1 if action == 2 else 0 # 0 and 1 are our labels

    # log the input and label to train later
    x_train.append(x)
    y_train.append(y)
    
    # do one step in our environment
    # print(env.step(action))
    new_state, reward, done, info, _ = env.step(action)
    observation = [new_state]
    rewards.append(reward)
    reward_sum += reward
    
    # end of an episode
    if done:
        print('At the end of episode', episode_nb, 'the total reward was :', reward_sum)
        
        # increment episode number
        episode_nb += 1
        
        # training
        model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, sample_weight=discount_rewards(rewards, gamma))
        model.save('./weights/testing_weight.keras')
        
        # Reinitialization
        x_train, y_train, rewards = [],[],[]
        observation = env.reset()
        reward_sum = 0
        prev_input = None

C:\Users\Tejas\anaconda3\envs\DeepLearning\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


1/1 [==============================] - 0s 35ms/step


C:\Users\Tejas\anaconda3\envs\DeepLearning\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 [==============================] - 0s 33ms/step


: 

: 

# Testing

In [6]:
import numpy as np
import gym

model = keras.models.load_model('./weights/og_weight_test.keras')

# gym initialization
env = gym.make("Pong-v4", render_mode="human")
observation = env.reset()
prev_input = None

# Macros
UP_ACTION = 2
DOWN_ACTION = 3

# Hyperparameters
gamma = 0.99

# initialization of variables used in the main loop
reward_sum = 0
episode_nb = 0

print(observation[0].shape)

while True:
    env.render()
    cur_input = prepro(observation[0])
    x = cur_input - prev_input if prev_input is not None else np.zeros(80 * 80)
    prev_input = cur_input
    
    # forward the policy network and sample action according to the proba distribution
    proba = model.predict(np.expand_dims(x, axis=1).T)
    action = UP_ACTION if np.random.uniform() < proba else DOWN_ACTION


    # do one step in our environment
    new_state, reward, done, info, _ = env.step(action)
    
    observation = [new_state]
    rewards.append(reward)
    reward_sum += reward
    
    # end of an episode
    if done:
        print('At the end of episode', episode_nb, 'the total reward was :', reward_sum)
        
        # increment episode number
        episode_nb += 1
        
        observation = env.reset()
        reward_sum = 0
        prev_input = None
        break

(210, 160, 3)
1/1 [==============================] - 0s 85ms/step


C:\Users\Tejas\anaconda3\envs\DeepLearning\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


1/1 [==============================] - 0s 22ms/step


C:\Users\Tejas\anaconda3\envs\DeepLearning\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 [==============================] - 0s 49ms/step


KeyboardInterrupt: 